In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px



In [ ]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
site_names =['CCAFS LC-40','VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40']
# Create a dash application

In [ ]:
spacex_df.head()

In [ ]:
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.Br(),
                                dcc.Dropdown(id='site-dropdown',
                                            options=[
                                                {'label': 'All Sites', 'value': 'ALL'},
                                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                            ],
                                            value= 'ALL',
                                            placeholder= "Select a Launch Site here",
                                            searchable=True
                                            ),
                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0',
                                                    100: '100'},
                                                value=[min_payload, max_payload]),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])


In [ ]:
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    filtered_df = spacex_df
    site_df = spacex_df[spacex_df['Launch Site'] == entered_site]
    site_count = site_df.groupby(['Launch Site', 'class']).size().reset_index(name = 'count')
   
    if entered_site == 'ALL':
        pie_fig = px.pie(spacex_df, values='class', names='Launch Site', title='Total Success Launches By Site')
        return pie_fig
    else:
        
        title = 'Total Success Launches for Site ' + str(entered_site)
        pie_fig = px.pie(site_count, values = 'count', names = 'class', title = title)
        return pie_fig
    

In [ ]:
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'),
              Input(component_id='payload-slider', component_property='value'))
def get_scatter_chart(entered_site, payload_slider):
    df = spacex_df
    slider_range = spacex_df[(spacex_df['Payload Mass (kg)'] >= min(payload_slider))\
                            & (spacex_df['Payload Mass (kg)'] <= max(payload_slider))]
    if entered_site == 'ALL':
       
        scatter_fig = px.scatter(slider_range, x='Payload Mass (kg)', y='class',\
                                 title= 'Correlation Between Payload and Success for all sites',\
                                 color='Booster Version Category')
        return scatter_fig
    else:
        site_scatter_df = slider_range[slider_range["Launch Site"] == entered_site]
        title = 'Correlation Between Payload and Success for' + entered_site
        scatter_fig = px.scatter(site_scatter_df, x='Payload Mass (kg)', y='class',\
                                 title= title,\
                                 color='Booster Version Category')
        return scatter_fig

In [ ]:
if __name__ == '__main__':
    app.run_server()